### Splitting the dataset

In [1]:
import csv  # reading csv files
import pandas as pd # data science
import numpy as np  # data science
from sklearn.model_selection import train_test_split    # splitting the data
import itertools    # getting a list of all words from a dataframe
from collections import Counter # for getting number of occurences of words
import regex as re
import ast

In [2]:
raw_data = pd.read_csv("data/sample_preprocessed_no_punct.csv")

/var/folders/gj/r8x5s5mn7jg12hs3c28wpg4c0000gn/T/ipykernel_18928/3667402737.py:1: DtypeWarning: Columns (1,2,3,13,17) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv("data/sample_preprocessed_no_punct.csv")


In [3]:
raw_data.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source
0,0.0,11467794,8027,9787368,nytimes.com,reliable,https://query.nytimes.com/gst/fullpage.html?re...,"['kashaaudrey', 'sission', 'lifelong', 'new', ...",2018-02-11 00:48:58.787555,2018-02-11 00:14:20.346838,2018-02-11 00:14:20.346871,"Paid Notice: Deaths KASHA, AUDREY SISSION",NaN,NaN,['KASHA AUDREY SISSION'],"KASHA--Audrey Sission, a lifelong New Yorker, ...",NaN,NaN,nytimes
1,1.0,4591569,8760,3009581,thinkprogress.org,political,https://thinkprogress.org/into-the-valley-of-d...,"['forward', 'light', 'brigadewa', 'man', 'dism...",2017-11-18T20:01:27.400599,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,"Into The Valley Of Death Rode The 600, Into Th...",NaN,NaN,[''],NaN,"#Climate Change, #Climate",NaN,NaN
2,2.0,6162754,2521,4251195,truthandaction.org,bias,http://www.truthandaction.org/woman-thrown-off...,"['woman', 'thrown', 'plane', 'said', 'hillari'...",2017-11-27T01:15:32.269834,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,Woman Thrown Off Plane When She Said Hillary i...,NaN,NaN,[''],NaN,NaN,NaN,NaN
3,3.0,2542246,2019,1769246,ecowatch.com,political,https://www.ecowatch.com/cuban-province-well-o...,"['cuban', 'provinc', 'well', 'way', 'num', 're...",2017-11-10T11:18:44.524042,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,Cuban Province Well on Its Way to 100% Renewab...,"Guest Contributor, Sierra Club, Common Dreams,...",NaN,"['featured', 'renewables', 'business', 'cuba']",President Obama’s recent announcement that he ...,NaN,NaN,NaN
4,4.0,3753783,4806,2429386,weeklystandard.com,political,http://www.weeklystandard.com/print/the-times-...,"['new', 'york', 'time', 'greet', 'deleg', 'fro...",2017-11-13T18:09:27.760857,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,The Times Repeats Itself,To The Scrapbook,NaN,['The Scrapbook'],The New York Times greeted delegates with a fr...,NaN,NaN,NaN


In [4]:
raw_data['content'][0]

"['kashaaudrey', 'sission', 'lifelong', 'new', 'yorker', 'born', 'decemb', 'num', 'num', 'die', 'june', 'num', 'num', 'daughter', 'late', 'helen', 'sissoncerussi', 'theodor', 'r', 'sisson', 'mother', 'late', 'matthew', 'p', 'kasha', 'sister', 'b', 'peter', 'cerussi', 'maxin', 'l', 'sisson', 'gloria', 'cerussi', 'gravesid', 'servic', 'friday', 'june', 'num', '11am', 'warwick', 'cemeteri', 'warwick', 'ny', 'contribut', 'memori', 'anim', 'welfar', 'group', 'choic', 'would', 'appreci', 'date']"

In [5]:
pattern = re.compile(r"\w+")
def string_to_list(s):
    return pattern.findall(s)
raw_data['content'] = raw_data['content'].apply(string_to_list)


In [6]:
len(raw_data)

650012

In [7]:
data = raw_data[raw_data['type'].isnull() == False]

THIS IS JUST TEMPORARY - HERE WE NEED TO DECIDE WHAT TO DO ABOUT THE FACT THAT NOT ALL ARTICLES ARE EITHER 'RELIABLE' OR 'FAKE'

In [8]:
#data = data[(data['type'] == 'fake') | (data['type'] == 'reliable')]

In [9]:
len(data)

621916

In [10]:
data.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source
0,0.0,11467794,8027,9787368,nytimes.com,reliable,https://query.nytimes.com/gst/fullpage.html?re...,"[kashaaudrey, sission, lifelong, new, yorker, ...",2018-02-11 00:48:58.787555,2018-02-11 00:14:20.346838,2018-02-11 00:14:20.346871,"Paid Notice: Deaths KASHA, AUDREY SISSION",NaN,NaN,['KASHA AUDREY SISSION'],"KASHA--Audrey Sission, a lifelong New Yorker, ...",NaN,NaN,nytimes
1,1.0,4591569,8760,3009581,thinkprogress.org,political,https://thinkprogress.org/into-the-valley-of-d...,"[forward, light, brigadewa, man, dismayd, tho,...",2017-11-18T20:01:27.400599,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,"Into The Valley Of Death Rode The 600, Into Th...",NaN,NaN,[''],NaN,"#Climate Change, #Climate",NaN,NaN
2,2.0,6162754,2521,4251195,truthandaction.org,bias,http://www.truthandaction.org/woman-thrown-off...,"[woman, thrown, plane, said, hillari, b]",2017-11-27T01:15:32.269834,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,Woman Thrown Off Plane When She Said Hillary i...,NaN,NaN,[''],NaN,NaN,NaN,NaN
3,3.0,2542246,2019,1769246,ecowatch.com,political,https://www.ecowatch.com/cuban-province-well-o...,"[cuban, provinc, well, way, num, renew, energi]",2017-11-10T11:18:44.524042,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,Cuban Province Well on Its Way to 100% Renewab...,"Guest Contributor, Sierra Club, Common Dreams,...",NaN,"['featured', 'renewables', 'business', 'cuba']",President Obama’s recent announcement that he ...,NaN,NaN,NaN
4,4.0,3753783,4806,2429386,weeklystandard.com,political,http://www.weeklystandard.com/print/the-times-...,"[new, york, time, greet, deleg, frontpag, scoo...",2017-11-13T18:09:27.760857,2018-02-07 23:39:33.852671,2018-02-07 23:39:33.852696,The Times Repeats Itself,To The Scrapbook,NaN,['The Scrapbook'],The New York Times greeted delegates with a fr...,NaN,NaN,NaN


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer=lambda x : x)
X = vectorizer.fit_transform(data['content'])
y = data['type']

Split data before representing our data as bag of words for naive bayes classifier in order to avoid statistics leaking into validation and testing dat.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=0)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=0)

In [ ]:
#X_train.head()

AttributeError: head not found

In [ ]:
X_train.shape

(466437, 1364774)

In [ ]:
len(y_train)

466437

In [ ]:
def create_bag_of_words(dataframe):
    # get all words appearing in reliable articles
    reliable_words = [word for list in list(itertools.chain(dataframe.loc[dataframe['type'] == 'reliable'].content)) for word in list]
    # get all words appearing in fake articles
    fake_words = [word for list in list(itertools.chain(dataframe.loc[dataframe['type'] == 'fake'].content)) for word in list]
    # count words in both lists
    count_reliable = Counter(reliable_words)
    count_fake = Counter(fake_words)
    return pd.DataFrame({'reliable': pd.Series(count_reliable), 'fake': pd.Series(count_fake)})

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train.toarray(), y_train)

: 

: 

In [ ]:
#X_train = create_bag_of_words(X_train)

In [ ]:
#X_train.head()

,reliable,fake
0,2.0,NaN
00,18.0,3.0
000bq,NaN,4.0
000deg,NaN,2.0
0012712kai,1.0,NaN


In [ ]:
#X_train.tail()

,reliable,fake
️truth,NaN,2.0
️ugggirl,NaN,1.0
️url,NaN,1.0
️wf,NaN,1.0
️win,NaN,1.0


In [ ]:
#X_train.iloc[50:100]

,reliable,fake
05,2.0,NaN
0500z,NaN,1.0
051date0239,1.0,NaN
054date1258,1.0,NaN
054date4145,1.0,NaN
0577n089112a,NaN,1.0
0577n089112b,NaN,1.0
05a4820,1.0,NaN
05pm,1.0,NaN
05th,NaN,2.0


Replace all NaN values with 0, and add 1 to all values in order for Naive Bayes to work correctly

In [ ]:
X_train = X_train.fillna(0)
X_train += 1

In [ ]:
X_train.head()

,reliable,fake
a,4343.0,1418.0
aa,228.0,110.0
aaa,347.0,127.0
aaaa,4.0,1.0
aaaaaaaah,2.0,1.0


In [ ]:
# define function, so we can apply the same steps to X_val and X_test
def process_for_nb(dataframe):
    dataframe = create_bag_of_words(dataframe)
    dataframe = dataframe.fillna(0)
    dataframe += 1
    return dataframe

In [ ]:
X_val = process_for_nb(X_val)
X_test = process_for_nb(X_test)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer(analyzer='word',
                             preprocessor=None,
                             stop_words='english',
                             max_features=100)

X_train_vectorized = vectorizer.fit_transform(X_train)
X_train_vectorized = X_train_vectorized.toarray()

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

ValueError: Found input variables with inconsistent numbers of samples: [437703, 147169]

In [ ]:
nb = MultinomialNB()
nb.fit(X_train,y_train)

ValueError: Input X contains NaN.
MultinomialNB does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

### Test

NOTE: Example of problem

In [ ]:
count = Counter(['hey', 'word', 'donald', 'donald'])
count

Counter({'hey': 1, 'word': 1, 'donald': 2})

In [ ]:
for word in count:
    print(count[word])

1
1
2


In [ ]:
count2 = Counter(['win', 'some', 'donald', 'donald'])
count + count2

Counter({'hey': 1, 'word': 1, 'donald': 4, 'win': 1, 'some': 1})

In [ ]:
pd.DataFrame({'reliable': pd.Series(count), 'fake': pd.Series(count2)})

,reliable,fake
donald,2.0,2.0
hey,1.0,NaN
some,NaN,1.0
win,NaN,1.0
word,1.0,NaN


In [ ]:
data.loc[data['type'] == 'reliable'].content

0         [kashaaudrey, sission, lifelong, new, yorker, ...
5         [market, news, thu, nov, num, num, num, am, ed...
19        [articl, last, tuesday, chang, manag, chelsea,...
24        [nathanael, chung, num, westfield, two, straig...
30        [area, larger, home, larger, lot, typic, sight...
                                ...                        
649993    [strategi, play, campaign, work, deni, oppon, ...
649998    [creat, powerpoint, present, document, go, fai...
650003    [human, weapon, grip, import, documentari, exa...
650009    [world, news, thu, nov, num, num, num, am, gmt...
650011    [q, law, govern, instal, solar, panel, new, yo...
Name: content, Length: 133693, dtype: object

In [ ]:
reliable_words = [word for list in (data.loc[data['type'] == 'reliable'].content) for word in list]
fake_words = [word for list in (data.loc[data['type'] == 'fake'].content) for word in list]
count_fake = Counter(fake_words)
count_reliable = Counter(reliable_words)
test = pd.DataFrame({'reliable': pd.Series(count_reliable), 'fake': pd.Series(count_fake)})

In [ ]:
test.head()

,reliable,fake
a,5665.0,1866.0
aa,280.0,145.0
aaa,476.0,159.0
aaaa,3.0,NaN
aaaaaaaah,1.0,NaN


In [ ]:
pattern = re.compile(r"\w+")
pattern.findall("'[donald', 'trump', 'donald]'")


['donald', 'trump', 'donald']

In [ ]:
Counter([word for list in list(itertools.chain(X_train.loc[X_train['type'] == 'reliable'].content)) for word in list])

Counter({'berri': 693,
         'dual': 287,
         'role': 10992,
         'author': 18631,
         'particip': 5035,
         'give': 21610,
         'justic': 8707,
         'awkwardli': 123,
         'divid': 2825,
         'person': 20268,
         'first': 63234,
         'half': 13320,
         'book': 22273,
         'deal': 18539,
         'year': 120563,
         'appoint': 4510,
         'function': 2517,
         'tradit': 8072,
         'academ': 2112,
         'historian': 1469,
         'second': 23774,
         'becom': 21161,
         'necessarili': 1695,
         'selfinterest': 142,
         'memoirist': 28,
         'unintend': 224,
         'consequ': 2148,
         'power': 21201,
         'episod': 3448,
         'commiss': 7656,
         'histori': 13529,
         'receiv': 48263,
         'dispassion': 64,
         'treatment': 4868,
         'seemingli': 1230,
         'minutia': 62,
         'period': 8293,
         'lengthi': 659,
         'attent': 6451,